## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import datetime

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

## Model

In [ ]:
# Functions for anomaly detection and evaluation
def sliding_window_std(df, column_name, window_size, threshold):
    results = []
    problem_vals = []  # List to store values that exceed the threshold
    problematic_rows = []  # List to store rows corresponding to problematic values
    for i in range(len(df)):
        # Calculate the window boundaries
        start_index = max(0, i - window_size // 2)
        end_index = min(len(df), i + window_size // 2 + 1)
        
        # Get the numbers within the window
        window = df[column_name].iloc[start_index:end_index]
        
        # Calculate the standard deviation of the window
        std_dev = window.std()
        
        # Append the result to the results list
        results.append((i, df.iloc[i][column_name], window.tolist(), std_dev))
        if len(results) >= 3:
            avg_prev_results = (results[-3][3] + results[-2][3]) / 2
            threshold_value = avg_prev_results + threshold

            if std_dev < threshold_value or std_dev > threshold_value :
                # Append the index and value to problem_vals
                row = finding_row_number(df[column_name],  window.iloc[-1])
                problematic_row = df.iloc[row]
                
                if removing_seasonal_points(problematic_row):
                    #problem_vals.append((i - window_size // 2, window.iloc[-1]))
                    df.at[row, 'predicted_anomalies'] = 1
                else:
                    pass
    return df
                

def finding_row_number(original_lst, anomaly_lst):
    i = -1
    for num in original_lst:
        i +=1
        if num == anomaly_lst:
            return i

def removing_seasonal_points(anomalous_row):
    a=anomalous_row["Time"]
    dt_obj = datetime.datetime.strptime(str(a), '%Y-%m-%d %H:%M:%S')
    if (dt_obj.minute== 0 and  dt_obj.second== 0):
        return False        
    else:
        return True

# specify the csv file
#df = pd.read_csv("")
#df['timestamps'] = pd.to_datetime(df['timestamps'], unit='s')

# Specify the column name containing the data
#column_name = ''

# Adjust window_size as needed
window_size = None

# Adjust threshold as needed
threshold = None

df['predicted_anomalies'] = 0

df=sliding_window_std(df, column_name, window_size, threshold)